In [1]:
from PIL import Image, ImageDraw, ImageChops
import numpy as np
import random as rnd
import copy

In [2]:
num_of_gens = 7000
image = Image.open("ob.jpg")

In [3]:
def draw(ind):
    imgdr = Image.new('RGB', (512, 512), (int(a), int(a), int(a)))
    drwim = ImageDraw.Draw(imgdr, 'RGBA')
    for ir in range(len(ind)):
        drwim.polygon([(ind[ir][0], ind[ir][1]), (ind[ir][2], ind[ir][3]), (ind[ir][4], ind[ir][5])], fill=(ind[ir][6], ind[ir][7], ind[ir][8], 125))
    return imgdr

In [4]:
def dr2img(ind):
    temp = copy.deepcopy(new_image)
    drwim = ImageDraw.Draw(temp, 'RGBA')
    drwim.polygon([(ind[0], ind[1]), (ind[2], ind[3]), (ind[4], ind[5])], fill=(ind[6], ind[7], ind[8], 125))
    return temp

In [5]:
def draw_obj(ind):
    imgdr = Image.new('RGB', (512, 512), (int(a), int(a), int(a)))
    drwim = ImageDraw.Draw(imgdr, 'RGBA')
    drwim.polygon([(ind[0], ind[1]), (ind[2], ind[3]), (ind[4], ind[5])], fill=(ind[6], ind[7], ind[8], 125))
    return imgdr

In [6]:
def fitness(image1, image2):
    return np.sum(np.absolute(np.array(ImageChops.difference(image1, image2))))

In [7]:
def mutate(obj):
    todo = rnd.randint(0, 1)
    if todo == 1:
        for i in range(6):
            obj[i] = obj[i] + rnd.randint(-150, 150)
    else:
        obj[6] = obj[6] + rnd.randint(-40, 40)
        obj[7] = obj[7] + rnd.randint(-40, 40)
        obj[8] = obj[8] + rnd.randint(-40, 40)

In [8]:
def generate(n):
    obj = []
    for i in range(n):
        temp = np.zeros(9, dtype=np.int32)
        for j in range(6):
            temp[j] = rnd.randint(0, 511)
        temp[6] = rnd.randint(0, 255)
        temp[7] = rnd.randint(0, 255)
        temp[8] = rnd.randint(0, 255)
        obj.append(temp)
    return obj

In [9]:
def cross(obj, obj1):
    temp = np.zeros(9, dtype=np.int32)
    for i in range(9):
        todo = rnd.randint(0, 1)
        if todo == 1:
            temp[i] = obj[i]
        else:
            temp[i] = obj1[i]
    return temp

In [10]:
def fitSort(val):
    return fitness(image, draw_obj(val))

In [11]:
a = np.sum(np.array(image)) // (512 * 512 * 3)
it = 0
population = generate(num_of_gens)
population.sort(key=fitSort)
population = population[0:3500]
fit_arr = []
for ind in range(len(population)):
    fit_arr.append(fitness(image, draw_obj(population[ind])))
new_image = draw(population)
new_image.save('ouput.png', 'PNG')
print(f'Generation: {it} Fitness: {fitness(image, draw(population))} Size: {len(population)}')
it += 1

Generation: 0 Fitness: 41336833 Size: 3500


In [12]:
while True:
    old_image = copy.deepcopy(new_image)
    fit = fitness(image, new_image)
    old_fit = copy.deepcopy(fit_arr)
    old_pop = copy.deepcopy(population)
    for ind in range(len(population)):
        tmp = copy.deepcopy(population[ind])
        mutate(population[ind])
        mut_image = dr2img(population[ind])
        if fit < fitness(image, mut_image):
            population[ind] = copy.deepcopy(tmp)
        else:
            fit = fitness(image, mut_image)
            new_image = mut_image
    print("Mutation completed!")
    for ind in range(len(population)):
        j = rnd.randint(0, 3499)
        k = rnd.randint(0, 3499)
        while j == k:
            j = rnd.randint(0, 3499)
            k = rnd.randint(0, 3499)
        crs = cross(population[j], population[k])
        fit3 = fitness(image, dr2img(crs))
        if fit3 < fit:
            population.append(crs)
            fit = fit3
            test_pop1 = copy.deepcopy(population)
            del test_pop1[j]
            test_pop2 = copy.deepcopy(population)
            del test_pop2[k]
            img1 = draw(test_pop1)
            img2 = draw(test_pop2)
            fit1 = fitness(image, img1)
            fit2 = fitness(image, img2)
            dlt = False
            if fit1 < fit:
                del population[j]
                new_image = img1
                fit = fit1
                dlt = True
            if fit2 < fit:
                if dlt and j < k:
                    del population[k-1]
                else:
                    del population[k]
                new_image = img2
                fit = fit2
    print("Crossover completed!")
    new_image.save('output.png', 'PNG')
    print(f'Generation: {it} Fitness: {fitness(image, new_image)} Size: {len(population)}')
    it += 1

Mutation completed!
Mutation completed!
Mutation completed!


KeyboardInterrupt: 